In [1]:
#!pip install pmlb
import pandas as pd
import numpy as np
from pmlb import fetch_data

In [2]:
# dataset for this demo
#https://github.com/EpistasisLab/penn-ml-benchmarks/tree/master/datasets/classification/GAMETES_Epistasis_2-Way_20atts_0.4H_EDM-1_1
gametes = fetch_data('GAMETES_Epistasis_2-Way_20atts_0.4H_EDM-1_1')
print(gametes.describe())

                N0           N1           N2           N3           N4  \
count  1600.000000  1600.000000  1600.000000  1600.000000  1600.000000   
mean      0.020625     0.261250     0.728750     0.889375     0.354375   
std       0.142170     0.475015     0.673949     0.702635     0.523416   
min       0.000000     0.000000     0.000000     0.000000     0.000000   
25%       0.000000     0.000000     0.000000     0.000000     0.000000   
50%       0.000000     0.000000     1.000000     1.000000     0.000000   
75%       0.000000     0.000000     1.000000     1.000000     1.000000   
max       1.000000     2.000000     2.000000     2.000000     2.000000   

                N5           N6           N7           N8           N9  ...  \
count  1600.000000  1600.000000  1600.000000  1600.000000  1600.000000  ...   
mean      0.791250     0.502500     0.728750     0.346875     0.761875  ...   
std       0.695864     0.615614     0.684994     0.540125     0.694062  ...   
min       0.00000

In [3]:
gametes.head()

,N0,N1,N2,N3,N4,N5,N6,N7,N8,N9,...,N11,N12,N13,N14,N15,N16,N17,P1,P2,target
0,0,0,2,1,1,0,0,2,0,1,...,2,1,0,0,0,0,0,0,1,1
1,0,0,1,0,0,1,1,0,0,1,...,0,1,0,0,0,0,0,0,0,1
2,0,0,0,1,0,2,0,0,0,0,...,1,1,0,0,2,2,0,0,0,1
3,0,1,0,2,0,1,0,2,0,0,...,1,1,0,1,1,0,1,1,0,1
4,0,0,1,1,0,0,0,1,1,0,...,1,0,0,1,0,0,1,1,0,1


In [4]:
X=gametes.drop('target',axis=1)

# features for A
A_feat_list = [
                'N10',
                'N11',
                'N12'
                ]

# features for C
C_feat_list = [
                'N13',
                'N14',
                'N15'
                ]
# y

y = gametes['target']

In [5]:
# test metaestimator alone

# without subset option
from tpot.builtins import MetaClassifier
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import cross_val_score, train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
metaest = MetaClassifier(estimator=GradientBoostingRegressor(random_state=42), A=A_feat_list, C=C_feat_list)
metaest.fit(X_train, y_train)
metaest.values_list

[['ternary',
  'ternary',
  'ternary',
  'ternary',
  'ternary',
  'ternary',
  'ternary',
  'ternary',
  'ternary',
  'ternary',
  'ternary',
  'ternary',
  'ternary',
  'ternary']]

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
X_train.head()

,N0,N1,N2,N3,N4,N5,N6,N7,N8,N9,N10,N11,N12,N13,N14,N15,N16,N17,P1,P2
820,0,0,0,2,1,1,0,1,1,0,0,1,1,1,0,0,1,1,1,1
567,0,0,0,0,0,0,0,0,0,0,0,2,2,0,1,0,0,2,0,1
355,0,1,0,1,1,1,1,1,0,1,0,1,2,0,0,0,1,0,0,0
1545,0,0,0,2,1,0,0,1,1,2,0,0,0,0,0,1,2,1,1,1
209,0,0,1,2,1,2,2,0,0,0,0,1,1,1,0,0,0,2,0,1


In [7]:
y_pred = metaest.predict(X_train)

In [8]:
from tpot.metrics import balanced_accuracy
b_acc = balanced_accuracy(y_true=y_train, y_pred=y_pred)
print(b_acc)

0.7294635022771905


In [9]:
print(metaest.score(X_test, y_test))

0.5492424242424242


In [10]:
# with subset option
# only one subset
metaest = MetaClassifier(estimator=GradientBoostingRegressor(random_state=42), 
                            A=A_feat_list, 
                            C=C_feat_list,
                            subset=['N1', 'N2', 'N3']) #only convert N1, N2 and N3
metaest.fit(X_train, y_train)
metaest.values_list

[['ternary', 'ternary', 'ternary']]

In [11]:
b_acc = balanced_accuracy(y_true=y_train, y_pred=y_pred)
print(b_acc)
print(metaest.score(X_test, y_test))

0.7294635022771905
0.7443181818181818


In [12]:
# with subset option
# a list of subsets
# 1st list should be C
# 2nd list should be subset of features for transfromation/adjust
subset_tuple = ((['N13', 'N14', 'N15'], ['N1']), # confounding variable N13-15 should adjust N1
               (['N13'], ['N2', 'N3']), # confounding variable N13 should adjust N2-3
               (['N14', 'N15'], ['N4', 'N5'])) # confounding variable N14-15 should adjust N4-5
metaest = MetaClassifier(estimator=GradientBoostingRegressor(random_state=42), 
                            A=A_feat_list, 
                            C=C_feat_list,
                            subset=subset_tuple) #only convert N1, N2 and N3
metaest.fit(X_train, y_train)
metaest.values_list

[['ternary'], ['ternary', 'ternary'], ['ternary', 'ternary']]

In [13]:
b_acc = balanced_accuracy(y_true=y_train, y_pred=y_pred)
print(b_acc)
print(metaest.score(X_test, y_test))

0.7294635022771905
0.7178030303030303


In [14]:
# metaestimator is passed when using it as a stand-alone estimator

In [15]:
# score is matched

In [16]:
# Test metaestimator with TPOT with config

from tpot import TPOTClassifier
subset_tuple = ((['N13', 'N14', 'N15'], ['N1']), # confounding variable N13-15 should adjust N1
               (['N13'], ['N2', 'N3']), # confounding variable N13 should adjust N2-3
               (['N14', 'N15'], ['N4', 'N5'])) # confounding variable N14-15 should adjust N4-5


tpot_config = {
    'tpot.builtins.FeatureSetSelector': {
        'subset_list': ['./subset_test.csv'],
        'sel_subset': [0, 1],
        'res_cols': [['N10', 'N11', 'N12', 'N13', 'N14', 'N15']]
    },
   'tpot.builtins.MetaClassifier': {
        
        'estimator': {
           'sklearn.ensemble.GradientBoostingRegressor': {
                'n_estimators': [100],
                'loss': ["ls", "lad", "huber", "quantile"],
                'learning_rate': [1e-3, 1e-2, 1e-1, 0.5, 1.],
                'max_depth': range(1, 11),
                'min_samples_split': range(2, 21),
                'min_samples_leaf': range(1, 21),
                'subsample': np.arange(0.05, 1.01, 0.05),
                'max_features': np.arange(0.05, 1.01, 0.05),
                'alpha': [0.75, 0.8, 0.85, 0.9, 0.95, 0.99]
            },

        },
       'A': [A_feat_list],
       'C': [C_feat_list],
       'subset': subset_tuple
    }
    
}


tpot = TPOTClassifier(generations=5, population_size=25, 
                        verbosity=3, cv=5,
                      config_dict=tpot_config,
                      template="FeatureSetSelector-Classifier",
                      scoring="balanced_accuracy",random_state=42)
tpot.fit(X_train, y_train)


2 operators have been imported by TPOT.


_pre_test decorator: _random_mutation_operator: num_test=0 not enough values to unpack (expected 2, got 1).
_pre_test decorator: _random_mutation_operator: num_test=1 not enough values to unpack (expected 2, got 1).
_pre_test decorator: _random_mutation_operator: num_test=2 not enough values to unpack (expected 2, got 1).
_pre_test decorator: _random_mutation_operator: num_test=3 not enough values to unpack (expected 2, got 1).
_pre_test decorator: _random_mutation_operator: num_test=4 not enough values to unpack (expected 2, got 1).
_pre_test decorator: _random_mutation_operator: num_test=5 not enough values to unpack (expected 2, got 1).
_pre_test decorator: _random_mutation_operator: num_test=6 not enough values to unpack (expected 2, got 1).
_pre_test decorator: _random_mutation_operator: num_test=7 not enough values to unpack (expected 2, got 1).
_pre_test decorator: _random_mutation_operator: num_test=8 too many values to unpack (expected 2).
_pre_test decorator: _random_mutation

_pre_test decorator: _random_mutation_operator: num_test=0 'N4'.
_pre_test decorator: _random_mutation_operator: num_test=0 'N4'.
_pre_test decorator: _random_mutation_operator: num_test=0 too many values to unpack (expected 2).
_pre_test decorator: _random_mutation_operator: num_test=0 not enough values to unpack (expected 2, got 1).
_pre_test decorator: _random_mutation_operator: num_test=0 too many values to unpack (expected 2).
Generation 5 - Current Pareto front scores:
-2	0.5184314946492548	MetaClassifier(FeatureSetSelector(input_matrix, FeatureSetSelector__res_cols=['N10', 'N11', 'N12', 'N13', 'N14', 'N15'], FeatureSetSelector__sel_subset=1, FeatureSetSelector__subset_list=./subset_test.csv), MetaClassifier__A=['N10', 'N11', 'N12'], MetaClassifier__C=['N13', 'N14', 'N15'], MetaClassifier__GradientBoostingRegressor__alpha=0.95, MetaClassifier__GradientBoostingRegressor__learning_rate=1.0, MetaClassifier__GradientBoostingRegressor__loss=lad, MetaClassifier__GradientBoostingRegress

TPOTClassifier(config_dict={'tpot.builtins.FeatureSetSelector': {'res_cols': [['N10',
                                                                               'N11',
                                                                               'N12',
                                                                               'N13',
                                                                               'N14',
                                                                               'N15']],
                                                                 'sel_subset': [0,
                                                                                1],
                                                                 'subset_list': ['./subset_test.csv']},
                            'tpot.builtins.MetaClassifier': {'A': [['N10',
                                                                    'N11',
                                                             

In [17]:
list(tpot.evaluated_individuals_.keys())[0]

"MetaClassifier(FeatureSetSelector(input_matrix, FeatureSetSelector__res_cols=['N10', 'N11', 'N12', 'N13', 'N14', 'N15'], FeatureSetSelector__sel_subset=1, FeatureSetSelector__subset_list=./subset_test.csv), MetaClassifier__A=['N10', 'N11', 'N12'], MetaClassifier__C=['N13', 'N14', 'N15'], MetaClassifier__GradientBoostingRegressor__alpha=0.75, MetaClassifier__GradientBoostingRegressor__learning_rate=0.5, MetaClassifier__GradientBoostingRegressor__loss=quantile, MetaClassifier__GradientBoostingRegressor__max_depth=8, MetaClassifier__GradientBoostingRegressor__max_features=0.55, MetaClassifier__GradientBoostingRegressor__min_samples_leaf=19, MetaClassifier__GradientBoostingRegressor__min_samples_split=18, MetaClassifier__GradientBoostingRegressor__n_estimators=100, MetaClassifier__GradientBoostingRegressor__subsample=0.4, MetaClassifier__subset=(['N14', 'N15'], ['N4', 'N5']))"

In [18]:
print(tpot.score(X_test, y_test))

0.5246422208172883
